# Instruction
1. define vacation details
2. Review weather and activity schedule
3. Implement the agent that generates day-by-day itenerary
4. Evaluate itenerary using set of criteria
5. Use the tools api
6. Implement a 2nd agent to revise the itenerary based on the feedback THINk -> ACT -> OBSERVEATION

In [14]:
import os, json, sys
from openai import OpenAI
from schemas import Traveler, TravelPlan, Weather, VacationInfo, Activity, ActivityRecommendation, ItineraryDay
from pprint import pprint
import datetime
from helper import do_chat_completion

client = OpenAI(base_url="https://openai.vocareum.com/v1", api_key="voc-1142391097160736424292768839a5d4b7a92.77641675")

MODEL = "gpt-4.1-mini"

In [2]:
VACATION_INFO_DICT = {
    "travelers": [
        {
            "name": "Yuri",
            "age": 30,
            "interests": ["tennis", "cooking", "comedy", "technology"]
        },
        {
            "name": "Hiro",
            "age": 25,
            "interests": ["reading", "music", "theatre", "art"] 
        }
    ],
    "destination": "AgentsVille",
    "date_of_arrival": "2025-06-10",
    "date_of_departure": "2025-06-12",
    "budget": 130
}

In [3]:
vacation_info = VacationInfo.model_validate(VACATION_INFO_DICT)

# Display the vacation info as a dictionary
pprint(vacation_info.model_dump())

# Check that VacationInfo contains the expected data
assert "travelers" in vacation_info.model_dump().keys(), "VacationInfo should contain 'travelers' key"
assert "destination" in vacation_info.model_dump().keys(), "VacationInfo should contain 'destination' key"
assert "date_of_arrival" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_arrival' key"
assert "date_of_departure" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_departure' key"
assert "budget" in vacation_info.model_dump().keys(), "VacationInfo should contain 'budget' key"
assert isinstance(vacation_info.travelers, list), "Travelers should be a list"
assert all(isinstance(traveler, Traveler) for traveler in vacation_info.travelers), "All travelers should be instances of Traveler class"
assert isinstance(vacation_info.date_of_arrival, datetime.date), "date_of_arrival should be a date"
assert isinstance(vacation_info.date_of_departure, datetime.date), "date_of_departure should be a date"
assert isinstance(vacation_info.budget, int), "budget should be an integer"

# If all assertions pass, print a success message
print("✅ VacationInfo data structure is valid!")

{'budget': 130,
 'date_of_arrival': datetime.date(2025, 6, 10),
 'date_of_departure': datetime.date(2025, 6, 12),
 'destination': 'AgentsVille',
 'travelers': [{'age': 30,
                'interests': [tennis, cooking, comedy, technology],
                'name': 'Yuri'},
               {'age': 25,
                'interests': [reading, music, theatre, art],
                'name': 'Hiro'}]}
✅ VacationInfo data structure is valid!


In [4]:
from mock_api import call_weather_api_mocked, call_activities_api_mocked
import pandas as pd
import json

pd.set_option("display.max_colwidth", None)  # Show full content in DataFrame cells

weather_for_dates = [
    call_weather_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
    for ts in pd.date_range(
        # TODO: Fill in the missing start and end dates from vacation_info
        start= vacation_info.date_of_arrival,
        end= vacation_info.date_of_departure,
        freq="D",
    )
]

weather_for_dates_df = pd.DataFrame(weather_for_dates)

activities_for_dates = [
    activity
    for ts in pd.date_range(
        # TODO: Fill in the missing start and end dates from vacation_info
        start= vacation_info.date_of_arrival,
        end= vacation_info.date_of_departure,
        freq="D",
    )
    for activity in call_activities_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
]

activities_for_dates_df = pd.DataFrame(activities_for_dates)


travel_plan_json_schema = json.dumps(TravelPlan.model_json_schema(), indent=2)
activities = json.dumps(activities_for_dates_df.to_dict(orient="records"), indent=2)
weather = json.dumps(weather_for_dates_df.to_dict(orient="records"), indent=2)

## Define Agent 1: Itenerary Agent ###

In [5]:
ITINERARY_AGENT_SYSTEM_PROMPT = f"""
You are a helpful travel assistant and should find the best itenerary plan for the traveler.

## Task

- Generate a day-by-day itenerary based on traveler's interests, the weather forecast and budget.
- Collect weather data for the duration of the traveller travel between the arrival and departure time in vacation info.
- Analyze the weather data and based on the traveller interests and find available activities, avoid outdoor-only activities on rainy days.
- When choosing activities the total cost should not exceed the budget
- Find the best activity for each day

## Output Format

Respond using two sections (ANALYSIS AND FINAL OUTPUT) in the following format:

    ANALYSIS:
    Provide your analysis data in the following format:
    1. **Weather**: show the expected wetaher based on the forecast data
    2. **Available Activities**: 
      - Show activites that are available for these dates based on the weather condition
      - Avoid activities if weather forcast is not good for outdoor
      - Expand variet of activities and try to plan different activites for each day
      - Make a list of activities based on traveller interest
    3. **Budget Check**: 
      - for each available activities find the budget information
      - try to provide optimized costs to stay under the budget of the traveller
    4. **Justification**: Provide explanation why these activities are suggested
    
    Do this for each day of itenrary.

    FINAL OUTPUT:
    ```json
    {travel_plan_json_schema}
    ```

    Provide the final output of the travel plan following TravelPlan schema

## Context

Here are the activites:
```json
{activities}
```

Here is the weather forcast:
```json
{weather}
```
"""

assert "TASK" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'TASK' section"
assert "OUTPUT FORMAT" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'OUTPUT FORMAT' section"

In [6]:
from helper import print_in_box
from agent import ChatAgent
from typing import Optional

class ItineraryAgent(ChatAgent):

    def get_itinerary(self, vacation_info: VacationInfo, model: str) -> TravelPlan:
        response = (self.chat(
            user_message=vacation_info.model_dump_json(indent=2),
            add_to_messages=False,
            model=model or self.model,
        ) or "").strip()

        print_in_box(response, "Raw Response")

        # Parse the response
        json_text = response.strip()

        if "```json" in json_text:
            json_text = json_text.split("```json")[1].split("```")[0].strip()

        try:
            travel_plan = TravelPlan.model_validate_json(json_text)
            return travel_plan
        except Exception as e:
            print("Error validating the following text as TravelPlan JSON:")
            print(json_text)
            raise

itinerary_agent = ItineraryAgent(client=client, model=MODEL, name="Itinerary_Agent", system_prompt = ITINERARY_AGENT_SYSTEM_PROMPT)


╔═════════════════════════════════════════[ Itinerary_Agent - System Prompt ]══════════════════════════════════════════╗
║ You are a helpful travel assistant and should find the best itenerary plan for the traveler.                         ║
║ ## Task                                                                                                              ║
║ - Generate a day-by-day itenerary based on traveler's interests, the weather forecast and budget.                    ║
║ - Collect weather data for the duration of the traveller travel between the arrival and departure time in vacation   ║
║ info.                                                                                                                ║
║ - Analyze the weather data and based on the traveller interests and find available activities, avoid outdoor-only    ║
║ activities on rainy days.                                                                                            ║
║ - When choosing activities th

In [7]:
travel_plan_1 = itinerary_agent.get_itinerary(
    vacation_info=vacation_info,
    model=MODEL,  # Optionally, you can change the model here
)

if travel_plan_1 is not None:
    print("✅ Initial itinerary generated successfully. Congratulations!")


╔══════════════════════════════════════════[ Itinerary_Agent - User Prompt ]═══════════════════════════════════════════╗
║ {                                                                                                                    ║
║   "travelers": [                                                                                                     ║
║     {                                                                                                                ║
║       "name": "Yuri",                                                                                                ║
║       "age": 30,                                                                                                     ║
║       "interests": [                                                                                                 ║
║         "tennis",                                                                                                    ║
║         "cooking",           

In [8]:
# evalute the result
from evalaute_agent import get_eval_results, eval_start_end_dates_match, eval_itinerary_satisfies_interests


In [9]:
get_eval_results(vacation_info, travel_plan_1, [eval_start_end_dates_match])

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_satisfies_interests],
)

✅ Traveler Yuri has a match with interest {cooking, tennis} at Serve & Savor: Tennis and Taste Luncheon
✅ Traveler Yuri has a match with interest {technology} at FutureTech Breakfast Meet-Up
✅ Traveler Yuri has a match with interest {technology} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Yuri has a match with interest {technology} at Tech & Film Fusion Night
✅ Traveler Hiro has a match with interest {art, reading} at AgentsVille Twilight Writing Escape
✅ Traveler Hiro has a match with interest {music} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Hiro has a match with interest {music} at Soundtrack Picnic: Lunchtime Movies & Melodies


EvaluationResults(success=True, failures=[], eval_functions=['eval_itinerary_satisfies_interests'])

In [24]:
ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT = """
You are a itenerary evaluator agent, your job is to evalaute the itenreray plan activities against the weather condition of that day.

## Task
You should identify if the planned acitivity is compatible with the weather condition;
if the activity is an outdoor activity, the weather condition should be suitable for that.
if there is not enough information to evalute the activity, you can assume that activity IS_COMPATIBLE.
Also evalaute the backup options provided in the activity plan 

## Output format

    REASONING:
    Provide step-by-step reasoning of your evalaution process.
    Highlight the key points of your evailuation on each acitivity

    FINAL ANSWER:
    [IS_COMPATIBLE, IS_INCOMPATIBLE]

## Examples
Example 1:
  - Activity: "Morning Groove Dance Party"
  - Weather: "rainy"
REASONING: "The acitivity is indoor, the weather outside is raniy"
FINAL ANSWER: IS_COMPATIBLE

Example 2: 
  - Activity: "Palette & Palate: Art Meets Cooking Experience"
  - Weather: "sunny"
REASONING: "The acitivity is indoor, the weather outside is sunny"
FINAL ANSWER: IS_INCOMPATIBLE
`
""".strip()

def eval_activities_and_weather_are_compatible(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that no outdoor-only activities are scheduled during inclement weather conditions.

    Args:
        vacation_info (dict): Contains the vacation details
        final_output (dict): Contains the itinerary details including daily activities and weather conditions

    Raises:
        AgentError: If any outdoor activities are scheduled during weather conditions that could ruin them
    """

    activities_that_are_incompatible = []

    for itinerary_day in final_output.itinerary_days:
        weather_condition = itinerary_day.weather.condition

        for activity_recommendation in itinerary_day.activity_recommendations:
            resp = do_chat_completion(
                messages=[
                    {
                        "role": "system",
                        "content": ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT,
                    },
                    {
                        "role": "user",
                        "content": f"Activity: {activity_recommendation.activity.name}\nDescription: {activity_recommendation.activity.description}\nWeather Condition: {weather_condition}",
                    },
                ],
                client=client,
                # This is a high-frequency use case, so we use a fast and cheap model.
                model=MODEL,
            )
    


            if "IS_COMPATIBLE" in (resp or ""):
                is_compatible = True
            elif "IS_INCOMPATIBLE" in (resp or ""):
                is_compatible = False
            else:
                raise RuntimeError(
                    f"Unexpected response from the model: {resp}. Expected 'IS_COMPATIBLE' or 'IS_INCOMPATIBLE'."
                )

            if is_compatible:
                print(
                    f"✅ Activity {activity_recommendation.activity.name} (on {itinerary_day.date}) and weather '{weather_condition}' are compatible."
                )

            else:
                activities_that_are_incompatible.append(
                    activity_recommendation.activity.name
                )
                print(
                    f"❌ Activity {activity_recommendation.activity.name} (on {itinerary_day.date}) and weather '{weather_condition}' are incompatible."
                )

    if activities_that_are_incompatible:
        raise AgentError(
            f"Activities that may be ruined by inclement weather: {activities_that_are_incompatible}"
        )

In [25]:
eval_results = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[
        eval_activities_and_weather_are_compatible
    ],
)
eval_results

✅ Activity Serve & Savor: Tennis and Taste Luncheon (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity FutureTech Breakfast Meet-Up (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity AgentsVille Twilight Writing Escape (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Tech Lunch & Learn: AI Frontiers (on 2025-06-11) and weather 'partly cloudy' are compatible.
✅ Activity Palette & Palate: Art Meets Cooking Experience (on 2025-06-11) and weather 'partly cloudy' are compatible.
✅ Activity Tech & Film Fusion Night (on 2025-06-12) and weather 'thunderstorm' are compatible.
✅ Activity Soundtrack Picnic: Lunchtime Movies & Melodies (on 2025-06-12) and weather 'thunderstorm' are compatible.


EvaluationResults(success=True, failures=[], eval_functions=['eval_activities_and_weather_are_compatible'])